In [ ]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=02f5b08e133de6f40051bad59a9a289a46dcb8f22dcbd0d2a9c653c4a601567a
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/Data Mining/Assignment 3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/Data Mining/Assignment 3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [ ]:
TrainCols = list(train_data.columns.values)
TestCols = list(test_data.columns.values)
print(TrainCols)
print(TestCols)

['CoverageField11A', 'CoverageField11B', 'CoverageField1A', 'CoverageField1B', 'CoverageField2A', 'CoverageField2B', 'CoverageField3A', 'CoverageField3B', 'CoverageField4A', 'CoverageField4B', 'CoverageField5A', 'CoverageField5B', 'CoverageField6A', 'CoverageField6B', 'Field10', 'Field11', 'Field7', 'Field8', 'Field9', 'GeographicField10A', 'GeographicField10B', 'GeographicField11A', 'GeographicField11B', 'GeographicField12A', 'GeographicField12B', 'GeographicField13A', 'GeographicField13B', 'GeographicField14A', 'GeographicField14B', 'GeographicField15A', 'GeographicField15B', 'GeographicField16A', 'GeographicField16B', 'GeographicField17A', 'GeographicField17B', 'GeographicField18A', 'GeographicField18B', 'GeographicField19A', 'GeographicField19B', 'GeographicField1A', 'GeographicField1B', 'GeographicField20A', 'GeographicField20B', 'GeographicField21A', 'GeographicField21B', 'GeographicField22A', 'GeographicField22B', 'GeographicField23A', 'GeographicField23B', 'GeographicField24A',

In [ ]:
# Seperate Target column from Train Data
Xtrain = train_data[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = train_data[['QuoteConversion_Flag']].copy()
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = test_data.copy()
print("Test Set shape:")
print(Xtest.shape)

Train Set shape:
(65000, 595)
(65000, 1)
Test Set shape:
(173836, 596)


In [ ]:
print(Xtest.columns)

Index(['CoverageField11A', 'CoverageField11B', 'CoverageField1A',
       'CoverageField1B', 'CoverageField2A', 'CoverageField2B',
       'CoverageField3A', 'CoverageField3B', 'CoverageField4A',
       'CoverageField4B',
       ...
       'PropertyField38_N', 'PropertyField38_Y', 'GeographicField63_ ',
       'GeographicField63_N', 'GeographicField63_Y', 'GeographicField64_CA',
       'GeographicField64_IL', 'GeographicField64_NJ', 'GeographicField64_TX',
       'GeographicField64'],
      dtype='object', length=596)


In [ ]:
Xtest.drop(["GeographicField64"],axis=1,inplace=True)


In [ ]:
sum(pd.isna(Ytrain).sum())

0

In [ ]:
Ytrain.replace('', np.NaN)

QuoteConversion_Flag
0                         0
1                         0
2                         0
3                         0
4                         0
...                     ...
64995                     0
64996                     0
64997                     0
64998                     0
64999                     0

[65000 rows x 1 columns]

Random Forest

In [ ]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(Xtrain, Ytrain)
rfc_predict=rfc.predict(Xtest)




In [ ]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=2)
rfc_random.fit(Xtrain, Ytrain)
grid_parm_rfc=rfc_random.best_params_
print("Best parameters:")
print(grid_parm_rfc)


Best parameters:
{'n_estimators': 110, 'min_samples_split': 80, 'max_depth': 7}


In [ ]:
#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(Xtrain,Ytrain)
rfc_predict = rfc.predict(Xtest)

In [ ]:
#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

=== All AUC Scores ===
[0.92299581 0.91915923 0.92728634 0.92788478 0.92469621]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9244044730261558


Exporting to then submit to Kaggle

In [ ]:
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(rfc.predict(Xtrain),columns=["Prediction"])

pred.head()

Prediction
0           0
1           0
2           0
3           0
4           0

In [ ]:
#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(rfc.predict_proba(Xtrain))

pred_Probability.head()

0         1
0  0.941908  0.058092
1  0.825331  0.174669
2  0.782653  0.217347
3  0.958476  0.041524
4  0.972512  0.027488

In [ ]:
#Print Classification Report
print(classification_report(Ytrain,pred))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92     52738
           1       0.99      0.23      0.38     12262

    accuracy                           0.85     65000
   macro avg       0.92      0.62      0.65     65000
weighted avg       0.88      0.85      0.82     65000



In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(rfc.predict(Xtest),columns=["Prediction"])
print(rfc.predict(Xtest))
pred.head()

[0 0 0 ... 0 0 0]


Prediction
0           0
1           0
2           0
3           0
4           0

In [ ]:
#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(rfc.predict_proba(Xtest))

pred_Probability.head()

0         1
0  0.943940  0.056060
1  0.818021  0.181979
2  0.844824  0.155176
3  0.943694  0.056306
4  0.830648  0.169352

In [ ]:
pred.to_csv('/gdrive/My Drive/Data Mining/Assignment 3/RandomForestOnlyResults2.csv')